In [113]:
import sympy as sp
import numpy as np
from scipy import optimize
import sys
sys.path.append('..')

from cge_modeling.sympy_tools import info_to_symbols, enumerate_indexbase, sub_all_eqs
from cge_modeling.production_functions import leontief, cobb_douglass
from cge_modeling.cge import compile_cge_to_numba, expand_compact_system, recursive_solve_symbolic, numba_linearize_cge_func
from cge_modeling.numba_tools import euler_approx

import pandas as pd

In [38]:
def symbol(name, *sectors):
    if sectors == ():
        return sp.Symbol(name, **default_assumptions)
    suffix = '_' + "_".join(sectors)
    return sp.Symbol(f'{name}{suffix}', **default_assumptions)


def symbols(name, value, sectors):
    return {symbol(name, sector): value for sector in sectors}

# Social Accounting Matrix

In [7]:
data = np.array([[0   , 0   , 0   , 1000, 2000, 4000],
                 [0   , 0   , 0   , 500 , 2000, 500 ],
                 [7000, 3000, 0   , 0   , 0   , 0   ],
                 [0   , 0   , 2000, 1000, 1000, 1000],
                 [0   , 0   , 2500, 2000, 3500, 3000],
                 [0   , 0   , 5500, 500 , 2500, 1000]])

cols = ['Ld','Kd', 'Men', 'Ag', 'Ind', 'Serv']
df = pd.DataFrame(data, columns=cols, index=cols, dtype='float64')

In [8]:
df

,Ld,Kd,Men,Ag,Ind,Serv
Ld,0.0,0.0,0.0,1000.0,2000.0,4000.0
Kd,0.0,0.0,0.0,500.0,2000.0,500.0
Men,7000.0,3000.0,0.0,0.0,0.0,0.0
Ag,0.0,0.0,2000.0,1000.0,1000.0,1000.0
Ind,0.0,0.0,2500.0,2000.0,3500.0,3000.0
Serv,0.0,0.0,5500.0,500.0,2500.0,1000.0


# Model

## Define variables, parameters, and sets

In [9]:
default_assumptions = {'real':True}
sectors = ['Ag', 'Ind', 'Serv']
i, j = [sp.Idx(name) for name in list('ij')]
index_dict = {
    i:sectors,
    j:sectors
}

In [27]:
var_info = [
    ('Y', (i, )),
    ('VA', (i, )),
    ('IC', (i, )),
    ('CIJ', (i, j)),
    ('C', (i, )),
    ('Ld', (i, )),
    ('Kd', (i, )),
    ('P', (i, )),
    ('P_VA', (i, )),
    ('P_IC', (i, )),
    ('U', ()),
    ('income', ()),
    ('r', ()),
    ('w', ()),
    ('walras_resid', ())
]

variables = info_to_symbols(var_info, default_assumptions)
Y, VA, IC, CIJ, C, Ld, Kd, P, P_VA, P_IC, U, income, r, w, walras_resid = variables


In [29]:
param_info = [
    ('psi_VA', (i, )),
    ('psi_IC', (i, )),
    ('psi_CIJ', (i, j)),
    ('alpha', (i, )),
    ('gamma', (i, )),
    ('A', (i, )),
    ('Ls', ()),
    ('Ks', ()),
    ('P_Ag_bar', ())
]

parameters = info_to_symbols(param_info, default_assumptions)
psi_VA, psi_IC, psi_CIJ, alpha, gamma, A, Ls, Ks, P_Ag_bar = parameters
    

## Define model equations

In [56]:
final_goods_block = leontief(Y, P, [IC, VA], [P_IC, P_VA], [psi_IC, psi_VA])
value_add_block = cobb_douglass(VA, P_VA, A, [Kd, Ld], [r, w], [alpha])

# Need to change how indexing is done in the intermediate block, because the model flips the indices for these equations
# So need to make some dummy variables
inter_names = ['P_IC', 'IC', 'P', 'CIJ', 'psi_CIJ', 'IC']
P_IC2, IC2, P2, CIJ2, psi_CIJ2, IC2 = [sp.IndexedBase(name, **default_assumptions) for name in inter_names]

intermediate_goods_block = [
    P_IC2[j] * IC2[j] - sp.Sum(P2[i] * CIJ2[i, j], (i, 0, 2)).doit(),
    CIJ2[i, j] - psi_CIJ2[i, j] * IC2[j],
]

other_equations = [
    # Income block
    income - w * Ls - r * Ks,

    # Household demand block
    U - sp.Product(C ** gamma, (i, 0, 2)).doit(),
    gamma * income - P * C,

    # Market Equlibrium
    Ls - sp.Sum(Ld, (i, 0, 2)).doit() - walras_resid,
    Ks - sp.Sum(Kd, (i, 0, 2)).doit(),
    C + sp.Sum(CIJ, (j, 0, 2)).doit() - Y,

    # Numeraire
    P.subs({i:0}) - P_Ag_bar
]

equations = final_goods_block + value_add_block + intermediate_goods_block + other_equations

# Calibration

In [57]:
symbols('P', 1, sectors)

{P_Ag: 1, P_Ind: 1, P_Serv: 1}

In [62]:
initial_values = {w:1, r:1, walras_resid:0}
initial_values.update(symbols('P', 1, sectors))
initial_values.update(symbols('P_VA', 1, sectors))
initial_values.update(symbols('P_IC', 1, sectors))

for sector in sectors:
    initial_values[symbol('Ld', sector)] = df.loc['Ld', sector]
    initial_values[symbol('Kd', sector)] = df.loc['Kd', sector]
    initial_values[symbol('VA', sector)] = ((initial_values[w] * initial_values[symbol('Ld', sector)] + 
                                            initial_values[r] * initial_values[symbol('Kd', sector)]) / initial_values[symbol('P_VA', sector)])
    for sector_j in sectors:
        initial_values[symbol('CIJ', sector, sector_j)] = df.loc[sector, sector_j]

In [63]:
initial_values

{w: 1,
 r: 1,
 walras_resid: 0,
 P_Ag: 1,
 P_Ind: 1,
 P_Serv: 1,
 P_VA_Ag: 1,
 P_VA_Ind: 1,
 P_VA_Serv: 1,
 P_IC_Ag: 1,
 P_IC_Ind: 1,
 P_IC_Serv: 1,
 Ld_Ag: 1000.0,
 Kd_Ag: 500.0,
 VA_Ag: 1500.0,
 CIJ_Ag_Ag: 1000.0,
 CIJ_Ag_Ind: 1000.0,
 CIJ_Ag_Serv: 1000.0,
 Ld_Ind: 2000.0,
 Kd_Ind: 2000.0,
 VA_Ind: 4000.0,
 CIJ_Ind_Ag: 2000.0,
 CIJ_Ind_Ind: 3500.0,
 CIJ_Ind_Serv: 3000.0,
 Ld_Serv: 4000.0,
 Kd_Serv: 500.0,
 VA_Serv: 4500.0,
 CIJ_Serv_Ag: 500.0,
 CIJ_Serv_Ind: 2500.0,
 CIJ_Serv_Serv: 1000.0}

In [64]:
long_system, named_variables, named_params = expand_compact_system(compact_equations=equations, 
                                                                   compact_variables=variables, 
                                                                   compact_params=parameters, 
                                                                   index_dict=index_dict,
                                                                   numeraire_dict={})
state_0 = recursive_solve_symbolic(long_system, initial_values)

### Check calibration

In [90]:
loss_funcs, system_funcs, ordered_inputs = compile_cge_to_numba(compact_equations=equations, 
                                                              compact_variables=variables, 
                                                              compact_params=parameters, 
                                                              index_dict=index_dict,
                                                              numeraire_dict={})

In [114]:
x0 = np.array([state_0[k] for k in named_variables], dtype=float)
theta0 = np.array([state_0[x] for x in named_params], dtype=float)

In [91]:
f_resid, f_grad, f_hess = loss_funcs
f_system, f_jac = system_funcs

The initial calibration should be an equlibrium, so the residuals should be zero, and all gradients of the loss function should be zero.

In [115]:
np.allclose(f_resid(x0, theta0), 0)

True

In [116]:
np.allclose(f_grad(x0, theta0), 0)

True

# Model Fitting

## Method 1: Use the Optimizer

In [107]:
capital_supply_shock = state_0.copy()
capital_supply_shock[Ks] = capital_supply_shock[Ks] * 1.1
calibrated_params = np.array([capital_supply_shock[x] for x in ordered_inputs[1]], dtype=float)

capital_shock_res = optimize.minimize(f_resid, x0, jac=f_grad, hess=f_hess, args=calibrated_params, method='trust-krylov')
assert capital_shock_res.success

In [109]:
labor_supply_shock = state_0.copy()
labor_supply_shock[Ls] = labor_supply_shock[Ls] * 1.1
calibrated_params = np.array([labor_supply_shock[x] for x in ordered_inputs[1]], dtype=float)

labor_shock_res = optimize.minimize(f_resid, x0, jac=f_grad, hess=f_hess, args=calibrated_params, method='trust-krylov')
assert labor_shock_res.success

In [110]:
scenario_df = pd.DataFrame(np.c_[x0, capital_shock_res.x, labor_shock_res.x], 
             index=[x.name for x in ordered_inputs[0]],
             columns=['initial', 'Ks_shock', 'Ls_shock'])

In [111]:
(scenario_df
    .assign(Ks_impact = lambda x: x.Ks_shock / x.initial - 1 ,
            Ls_impact = lambda x: x.Ls_shock / x.initial - 1)
    .replace({np.inf:0})
    .loc[:, ['Ks_impact', 'Ls_impact']]
    .applymap(lambda x: f'{x:0.3%}'))

,Ks_impact,Ls_impact
Y_Ag,3.166%,6.603%
Y_Ind,3.175%,6.591%
Y_Serv,2.542%,7.245%
VA_Ag,3.166%,6.603%
VA_Ind,3.175%,6.591%
VA_Serv,2.542%,7.245%
IC_Ag,3.166%,6.603%
IC_Ind,3.175%,6.591%
IC_Serv,2.542%,7.245%
CIJ_Ag_Ag,3.166%,6.603%


## Linear Approximation with Euler Iterations

In [112]:
f_dX = numba_linearize_cge_func(equations, variables, parameters, index_dict)

In [133]:
capital_supply_shock = state_0.copy()
capital_supply_shock[Ks] = capital_supply_shock[Ks] * 1.1
theta_capital = np.array([capital_supply_shock[x] for x in ordered_inputs[1]], dtype=float)

labor_supply_shock = state_0.copy()
labor_supply_shock[Ls] = labor_supply_shock[Ls] * 1.1
theta_labor = np.array([labor_supply_shock[x] for x in ordered_inputs[1]], dtype=float)

capital_shock_scenario = euler_approx(f_dX, x0, theta0, theta_capital, 100000)
labor_shock_scenario = euler_approx(f_dX, x0, theta0, theta_labor, 100000)

In [134]:
n_params = len(named_params)
scenario_df_linear = pd.DataFrame(np.c_[x0, capital_shock_scenario[:-n_params], labor_shock_scenario[:-n_params]], 
                                  index=[x.name for x in ordered_inputs[0]],
                                  columns=['initial', 'Ks_shock', 'Ls_shock'])

In [135]:
(scenario_df_linear
    .assign(Ks_impact = lambda x: x.Ks_shock / x.initial - 1 ,
            Ls_impact = lambda x: x.Ls_shock / x.initial - 1)
    .replace({np.inf:0, -np.inf:0})
    .loc[:, ['Ks_impact', 'Ls_impact']]
    .applymap(lambda x: f'{x:0.3%}'))

,Ks_impact,Ls_impact
Y_Ag,3.166%,6.603%
Y_Ind,3.175%,6.591%
Y_Serv,2.542%,7.245%
VA_Ag,3.166%,6.603%
VA_Ind,3.175%,6.591%
VA_Serv,2.542%,7.245%
IC_Ag,3.166%,6.603%
IC_Ind,3.175%,6.591%
IC_Serv,2.542%,7.245%
CIJ_Ag_Ag,3.166%,6.603%


In [128]:
scenario_df - scenario_df_linear

,initial,Ks_shock,Ls_shock
Y_Ag,0.0,-0.006089,-0.006066
Y_Ind,0.0,-0.013250,-0.013302
Y_Serv,0.0,-0.010537,-0.010529
VA_Ag,0.0,-0.001814,-0.001799
VA_Ind,0.0,-0.004815,-0.004836
VA_Serv,0.0,-0.004985,-0.004979
IC_Ag,0.0,-0.004257,-0.004229
IC_Ind,0.0,-0.008428,-0.008462
IC_Serv,0.0,-0.005541,-0.005541
CIJ_Ag_Ag,0.0,-0.001214,-0.001187
